In [1]:
import pandas as pd
import datetime
import calendar
import statsmodels.formula.api as sm

In [2]:
features_df = pd.read_csv("features.csv")
stores_df = pd.read_csv("stores.csv")
train_df = pd.read_csv("train.csv")

In [3]:
seasons_dict = {
    1:"Winter",
    2:"Winter",
    3:"Spring",
    4:"Spring",
    5:"Spring",
    6:"Summer",
    7:"Summer",
    8:"Summer",
    9:"Fall",
    10:"Fall",
    11:"Fall",
    12:"Winter"
}

In [4]:
master_df = train_df.merge(stores_df, on='Store', how='left')
master_df = master_df.merge(features_df, on=['Store', 'Date'], how='left')

In [5]:
master_df['MarkDown1'] = master_df['MarkDown1'].fillna(0)
master_df['MarkDown2'] = master_df['MarkDown2'].fillna(0)
master_df['MarkDown3'] = master_df['MarkDown3'].fillna(0)
master_df['MarkDown4'] = master_df['MarkDown4'].fillna(0)
master_df['MarkDown5'] = master_df['MarkDown5'].fillna(0)
master_df['isHoliday'] = master_df['IsHoliday_x']
master_df = master_df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
master_df['Date'] = pd.to_datetime(master_df['Date'], format='%Y-%m-%d')
master_df['Week_Number'] = master_df['Date'].dt.week + 1
master_df['Quarter'] = master_df['Date'].dt.quarter
master_df['Month'] = master_df['Date'].dt.month.apply(lambda x: calendar.month_abbr[x])
master_df['Season'] = (master_df['Date'].apply(lambda dt: (dt.month%12 + 3)//3)).map(seasons_dict)

In [6]:
master_df.head()

,Store,Dept,Date,Weekly_Sales,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,isHoliday,Week_Number,Quarter,Month,Season
0,1,1,2010-02-05,24924.50,A,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,6,1,Feb,Winter
1,1,1,2010-02-12,46039.49,A,151315,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True,7,1,Feb,Winter
2,1,1,2010-02-19,41595.55,A,151315,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False,8,1,Feb,Winter
3,1,1,2010-02-26,19403.54,A,151315,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False,9,1,Feb,Winter
4,1,1,2010-03-05,21827.90,A,151315,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,False,10,1,Mar,Winter


In [7]:
master_df = master_df.join(pd.get_dummies(master_df['Quarter'], prefix='Quarter'))

In [8]:
master_df.head()

,Store,Dept,Date,Weekly_Sales,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,...,Unemployment,isHoliday,Week_Number,Quarter,Month,Season,Quarter_1,Quarter_2,Quarter_3,Quarter_4
0,1,1,2010-02-05,24924.50,A,151315,42.31,2.572,0.0,0.0,...,8.106,False,6,1,Feb,Winter,1,0,0,0
1,1,1,2010-02-12,46039.49,A,151315,38.51,2.548,0.0,0.0,...,8.106,True,7,1,Feb,Winter,1,0,0,0
2,1,1,2010-02-19,41595.55,A,151315,39.93,2.514,0.0,0.0,...,8.106,False,8,1,Feb,Winter,1,0,0,0
3,1,1,2010-02-26,19403.54,A,151315,46.63,2.561,0.0,0.0,...,8.106,False,9,1,Feb,Winter,1,0,0,0
4,1,1,2010-03-05,21827.90,A,151315,46.50,2.625,0.0,0.0,...,8.106,False,10,1,Mar,Winter,1,0,0,0


In [14]:
result = sm.ols(formula="Weekly_Sales ~ isHoliday + Fuel_Price + Temperature + Size + MarkDown1 + MarkDown2 + MarkDown3 + MarkDown4 + MarkDown5 + CPI + Unemployment + Week_Number", data=master_df[(master_df['Store'] == 2)]).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:           Weekly_Sales   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.315
Date:                Wed, 18 Jul 2018   Prob (F-statistic):            0.00786
Time:                        14:54:59   Log-Likelihood:            -1.2106e+05
No. Observations:               10238   AIC:                         2.421e+05
Df Residuals:                   10226   BIC:                         2.422e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -1.086e-06   2.95e-0

In [11]:
master_df.describe()

,Store,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Week_Number,Quarter,Quarter_1,Quarter_2,Quarter_3,Quarter_4
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123,136727.915739,60.090059,3.361027,2590.074819,879.974298,468.087665,1083.132268,1662.772385,171.201947,7.960289,26.826762,2.482767,0.231331,0.272275,0.278692,0.217703
std,12.785297,30.492054,22711.183519,60980.583328,18.447931,0.458515,6052.385934,5084.538801,5528.873453,3894.529945,4207.629321,39.159276,1.863296,14.151887,1.071341,0.421684,0.445131,0.448356,0.412685
min,1.000000,1.000000,-4988.940000,34875.000000,-2.060000,2.472000,0.000000,-265.760000,-29.100000,0.000000,0.000000,126.064000,3.879000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,18.000000,2079.650000,93638.000000,46.680000,2.933000,0.000000,0.000000,0.000000,0.000000,0.000000,132.022667,6.891000,15.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,22.000000,37.000000,7612.030000,140167.000000,62.090000,3.452000,0.000000,0.000000,0.000000,0.000000,0.000000,182.318780,7.866000,27.000000,2.000000,0.000000,0.000000,0.000000,0.000000
75%,33.000000,74.000000,20205.852500,202505.000000,74.280000,3.738000,2809.050000,2.200000,4.540000,425.290000,2168.040000,212.416993,8.572000,39.000000,3.000000,0.000000,1.000000,1.000000,0.000000
max,45.000000,99.000000,693099.360000,219622.000000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,227.232807,14.313000,53.000000,4.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
result = sm.ols(formula="Weekly_Sales ~ isHoliday + Fuel_Price + Temperature + Size + MarkDown1 + MarkDown2 + MarkDown3 + MarkDown4 + MarkDown5 + CPI + Unemployment + Week_Number", data=master_df[(master_df['Store'] == 1)]).fit()
print(result.summary())

result = sm.ols(formula="Weekly_Sales ~ isHoliday + Fuel_Price + Temperature + Size + MarkDown1 + MarkDown2 + MarkDown3 + MarkDown4 + MarkDown5 + CPI + Unemployment + Week_Number", data=master_df[(master_df['Store'] == 2)]).fit()
print(result.summary())

result = sm.ols(formula="Weekly_Sales ~ isHoliday + Fuel_Price + Temperature + Size + MarkDown1 + MarkDown2 + MarkDown3 + MarkDown4 + MarkDown5 + CPI + Unemployment + Week_Number", data=master_df[(master_df['Store'] == 3)]).fit()
print(result.summary())

result = sm.ols(formula="Weekly_Sales ~ isHoliday + Fuel_Price + Temperature + Size + MarkDown1 + MarkDown2 + MarkDown3 + MarkDown4 + MarkDown5 + CPI + Unemployment + Week_Number", data=master_df[(master_df['Store'] == 4)]).fit()
print(result.summary())

result = sm.ols(formula="Weekly_Sales ~ isHoliday + Fuel_Price + Temperature + Size + MarkDown1 + MarkDown2 + MarkDown3 + MarkDown4 + MarkDown5 + CPI + Unemployment + Week_Number", data=master_df[(master_df['Store'] == 5)]).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:           Weekly_Sales   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.290
Date:                Wed, 18 Jul 2018   Prob (F-statistic):              0.223
Time:                        14:56:07   Log-Likelihood:            -1.1933e+05
No. Observations:               10244   AIC:                         2.387e+05
Df Residuals:                   10232   BIC:                         2.388e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          1.751e-07    2.2e-0

                            OLS Regression Results                            
Dep. Variable:           Weekly_Sales   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     4.334
Date:                Wed, 18 Jul 2018   Prob (F-statistic):           1.69e-06
Time:                        14:56:08   Log-Likelihood:            -1.2191e+05
No. Observations:               10272   AIC:                         2.438e+05
Df Residuals:                   10260   BIC:                         2.439e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -3.653e-06   2.95e-0

In [19]:
master_df[(master_df['Store'] == 1)].describe()

,Store,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Week_Number,Quarter,Quarter_1,Quarter_2,Quarter_3,Quarter_4
count,10244.0,10244.000000,10244.000000,10244.0,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000
mean,1.0,44.391742,21710.543621,151315.0,68.224463,3.218811,2886.877467,873.291894,436.207316,1340.570529,1794.852645,215.995525,7.610918,26.840785,2.484088,0.231941,0.270695,0.278700,0.218665
std,0.0,29.867247,27748.945511,0.0,14.203190,0.425663,5503.673703,4499.776135,4693.473924,3984.508270,3100.783515,4.335980,0.382613,14.167845,1.072879,0.422092,0.444341,0.448381,0.413361
min,1.0,1.000000,-863.000000,151315.0,35.400000,2.514000,0.000000,0.000000,0.000000,0.000000,0.000000,210.337426,6.573000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,20.000000,3465.622500,151315.0,57.790000,2.759000,0.000000,0.000000,0.000000,0.000000,0.000000,211.531248,7.348000,15.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,38.000000,10289.375000,151315.0,69.640000,3.290000,0.000000,0.000000,0.000000,0.000000,0.000000,215.459905,7.787000,27.000000,2.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,72.000000,31452.957500,151315.0,80.480000,3.594000,4640.650000,12.000000,8.110000,983.650000,3400.210000,220.636902,7.838000,39.000000,3.000000,0.000000,1.000000,1.000000,0.000000
max,1.0,99.000000,203670.470000,151315.0,91.650000,3.907000,34577.060000,46011.380000,55805.510000,32403.870000,20475.320000,223.444251,8.106000,53.000000,4.000000,1.000000,1.000000,1.000000,1.000000
